In [219]:
import pandas as pd
import re

In [220]:
log = pd.read_csv('ml-latest-small\\visit_log.csv', sep=';')
urls = pd.read_csv('ml-latest-small\\URLs.txt')
rates = pd.read_csv('ml-latest-small\\ratings.csv')

In [229]:
def f_source_type (ts, reg):
    if ts == 'yandex' or ts == 'google':
        return 'organic'
    elif (ts == 'paid' or ts == 'email') and reg == 'Russia':
        return 'ad'
    elif (ts == 'paid' or ts == 'email') and reg != 'Russia':
        return 'other'
    else: 
        return ts

In [230]:
#Для датафрейма log создайте столбец source_type по определенным правилам

log['source_type'] = log.apply(lambda x: f_source_type(x['traffic_source'], x['region']), axis = 1)

log.tail()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad
18937,1550094323,b3b634f824,https://host.ru/39fa884393666d45fc,Russia,7d27a58fb8,email,ad


In [223]:
#Необходимо отфильтровать файл URLs.txt по адресам страниц с текстами новостей

urls[urls.url.str.contains(r'/\d{8}-')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


In [224]:
#Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок

count = rates.groupby('userId').count().reset_index()
timestamps = rates.groupby('userId').timestamp.agg(
    maxts = 'max',
    mints = 'min').reset_index()
df = count.merge(timestamps, how='outer')
df['lifetime'] = (df['maxts'] - df['mints'])
df.query('rating > 100')

,userId,movieId,rating,timestamp,maxts,mints,lifetime
0,1,232,232,232,965719662,964980499,739163
3,4,216,216,216,1007574542,945078428,62496114
5,6,314,314,314,845556915,845553109,3806
6,7,152,152,152,1176181731,1106635416,69546315
9,10,140,140,140,1455619275,1455301553,317722
...,...,...,...,...,...,...,...
604,605,221,221,221,1277177012,1277094764,82248
605,606,1115,1115,1115,1368460577,1171228846,197231731
606,607,187,187,187,997847387,963078392,34768995
607,608,831,831,831,1189563917,1117161730,72402187


In [225]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [226]:
#Таблица с тремя типами выручки для каждого client_id без указания адреса клиента

rzd_auto = rzd.merge(auto, how='outer', )
rzd_auto_air = rzd_auto.merge(air, how='outer')

rzd_auto_air.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [227]:
#Аналогичная таблица по типам выручки с указанием адреса клиента

address_rzd_auto_air = client_base.merge(rzd_auto_air, how='outer') 

address_rzd_auto_air.fillna(0)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
